In [ ]:
import os
import numpy as np

DATA_DIR="Data/"

train_X=np.load(os.path.join(DATA_DIR,"PP_train_X.npy"))
train_y=np.load(os.path.join(DATA_DIR,"PP_train_y.npy"))
test_X=np.load(os.path.join(DATA_DIR,"PP_test_X.npy"))
test_y=np.load(os.path.join(DATA_DIR,"PP_test_y.npy"))

In [ ]:
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_accuracy, crf_viterbi_accuracy
from keras.optimizers import SGD
from keras.callbacks import TensorBoard, ModelCheckpoint, CSVLogger
from datetime import datetime


MAX_LEN=69
n_words=4169
n_tags=5

def trainM(model, mod_dir):
	MODEL_DIR=mod_dir
	N_EPOCHS=50
	B_SIZE=32

	if not os.path.exists(MODEL_DIR):
		os.makedirs(MODEL_DIR)

	chkpt_path=os.path.join(MODEL_DIR,"{epoch:02d}-{val_crf_accuracy:.4f}.hdf5")
	checkpointer = ModelCheckpoint(chkpt_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

	log_path=os.path.join(MODEL_DIR,"metrics.log")
	csv_logger = CSVLogger(log_path)

	tensorboard=TensorBoard(log_dir=MODEL_DIR)

	

	history = model.fit(train_X, 
						train_y, 
						batch_size=B_SIZE, 
						epochs=N_EPOCHS,
						validation_split=0.1, 
						verbose=1,
						callbacks=[checkpointer,csv_logger,tensorboard])

In [ ]:
from keras.optimizers import Adadelta,Adagrad, Adam, Adamax, Nadam, RMSprop, SGD
MODEL_NAME="LSTM-CRF"

opt=[]
opt.append(("Adadelta",Adadelta()))
opt.append(("Adagrad",Adagrad()))
opt.append(("Adam",Adam()))
opt.append(("Adamax",Adamax()))
opt.append(("Nadam",Nadam()))
opt.append(("rmsprop",RMSprop()))
opt.append(("SGD",SGD()))
opt.append(("SGDNesterov",SGD(nesterov=True)))

for c_opt, optimizer in opt:
	try:
		mod_dir="Models/"+MODEL_NAME+"/"+c_opt+"_"+datetime.now().strftime("%Y%m%d-%H%M%S")
		input = Input(shape=(MAX_LEN,))
		model = Embedding(input_dim=n_words + 1, output_dim=20,
						  input_length=MAX_LEN, mask_zero=True)(input)
		model = Bidirectional(LSTM(units=50, return_sequences=True,
								   recurrent_dropout=0.1))(model) 
		model = TimeDistributed(Dense(50, activation="relu"))(model)
		crf = CRF(n_tags)
		out = crf(model)

		model = Model(input, out)


		model.compile(optimizer=optimizer, loss=crf_loss, metrics=[crf_accuracy])
		model.summary()
		trainM(model,mod_dir)
	except Exception as e:
		print(e)